In [1]:
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2

# defining the answer key which maps the question number to the correct answer
ANSWER_KEY = {0: 1, 1: 4, 2: 0, 3: 3, 4: 1}

# loading the image, converting it to grayscale, blurring it slightly, then finding edges
image = cv2.imread('test_01.png')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)
edged = cv2.Canny(blurred, 75, 200)

# finding contours in the edge map, then initializing the contour that corresponds to the document
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
docCnt = None

# ensuring that at least one contour was found
if len(cnts) > 0:
    # sorting the contours according to their size in descending order
    cnts = sorted(cnts, key=cv2.contourArea, reverse=True)

    # looping over the sorted contours
    for c in cnts:
        # approximating the contour
        peri = cv2.arcLength(c, True)
        approx = cv2.approxPolyDP(c, 0.02 * peri, True)

        # if our approximated contour has four points, then we can assume we have found the paper
        if len(approx) == 4:
            docCnt = approx
            break

# applying a four point perspective transformation to both the original image and grayscale image to obtain a 
# top-down birds eye view of the paper
paper = four_point_transform(image, docCnt.reshape(4, 2))
warped = four_point_transform(gray, docCnt.reshape(4, 2))

# applying Otsu's thresholding method to binarize the warped piece of paper
thresh = cv2.threshold(warped, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

# finding contours in the thresholded image, then initializing the list of contours that correspond to questions
cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
questionCnts = []

# looping over the contours
for c in cnts:
    # computing the bounding box of the contour, then using the bounding box to derive the aspect ratio
    (x, y, w, h) = cv2.boundingRect(c)
    ar = w / float(h)

    # in order to label the contour as a question, region should be sufficiently wide, sufficiently tall,
    # and have an aspect ratio approximately equal to 1
    if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
        questionCnts.append(c)

# sorting the question contours top-to-bottom, then initializing the total number of correct answers
questionCnts = contours.sort_contours(questionCnts,method="top-to-bottom")[0]
correct = 0

# each question has 5 possible answers, to loop over the question in batches of 5
for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
    # sorting the contours for the current question from left to right, then initializing the index of the
    # bubbled answer
    cnts = contours.sort_contours(questionCnts[i:i + 5])[0]
    bubbled = None

    # loop over the sorted contours
    for (j, c) in enumerate(cnts):
        # construct a mask that reveals only the current
        # "bubble" for the question
        mask = np.zeros(thresh.shape, dtype="uint8")
        cv2.drawContours(mask, [c], -1, 255, -1)

        # apply the mask to the thresholded image, then
        # count the number of non-zero pixels in the
        # bubble area
        mask = cv2.bitwise_and(thresh, thresh, mask=mask)
        total = cv2.countNonZero(mask)

        # if the current total has a larger number of total
        # non-zero pixels, then we are examining the currently
        # bubbled-in answer
        if bubbled is None or total > bubbled[0]:
            bubbled = (total, j)

    # initialize the contour color and the index of the
    # *correct* answer
    color = (0, 0, 255)
    k = ANSWER_KEY[q]

    # check to see if the bubbled answer is correct
    if k == bubbled[1]:
        color = (0, 255, 0)
        correct += 1

    # draw the outline of the correct answer on the test
    cv2.drawContours(paper, [cnts[k]], -1, color, 3)

# grab the test taker
score = (correct / 5.0) * 100
print("[INFO] score: {:.2f}%".format(score))
cv2.putText(paper, "{:.2f}%".format(score), (10, 30),
    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("Exam", paper)
cv2.waitKey(0)

[INFO] score: 80.00%


Optical Mark Recognition, hoặc OMR đơn giản là quá trình tự dộng phân tích tài liệu được con người tạo ra và  làm sáng tỏ kết quả

In [26]:
# import the necessary packages
from imutils.perspective import four_point_transform
from imutils import contours
import numpy as np
import argparse
import imutils
import cv2

In [ ]:
cv2.destroyAllWindows()

In [54]:
# define the answer key 
# which maps 
# the question number to the correct answer
ANSWER_KEY = {0: 1, 
              1: 4, 
              2: 0, 
              3: 3, 
              4: 1}

In [55]:
pth="/home/anhp/Documents/coding/learnOpenCV/BubbleSheetScanner/Bubble-Sheet-Scanner/OMR Sheets/test_02.png"

xu ly anh dau vao

In [60]:
# load the image, 
# convert it to grayscale, 
# blur it slightly, 
# then find edges
image	= cv2.imread(pth)                           # tai anh vao
gray 	= cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)   # chuyen doi sang gray
blurred = cv2.GaussianBlur(gray, (5, 5), 0)         # lam mo anh
edged 	= cv2.Canny(blurred, 75, 200)               # tim canh cua doi tuong trong anh

cv2.imshow("Image", edged)
cv2.waitKey(0)
cv2.destroyAllWindows()

 Lưu ý rằng cách các cạnh của tài liệu cần được xác định rõ ràng, 
 với cả bốn đỉnh của ảnh _ bài trắc nghiệm scan_ 
 
 Việc này rất quan trọng trong bước tiếp theo của chúng ta, 
 vì chúng ta sẽ sử dụng nó như một điểm đánh dấu 
 để kéo giãn và xóa hiệu ứng mắt chim:

In [ ]:
# find contours in the edge map, 
# then initialize the contour 
# that corresponds to the document
cnts = cv2.findContours(edged.copy(), 
                        cv2.RETR_EXTERNAL,
						cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)

#cnts = cnts[0] if imutils.is_cv2() else cnts[1]

docCnt = None

output = image.copy()  #-----------------------------------------------


In [61]:

# ensure that at least one contour was found
if len(cnts) > 0:
	# sort the contours according to their size 
 	# in descending order
	cnts = sorted(cnts, key=cv2.contourArea, reverse=True)
	
 	# loop over the sorted contours
  	# ở mỗi contour chúng ta sẽ tìm các góc của contours sau khi approximated. 
	for c in cnts:
		# approximate the contour
		peri 	= cv2.arcLength(c, True)
		approx 	= cv2.approxPolyDP(c, 0.02 * peri, True)
  
		# if our approximated contour has four points,
		# then we can assume we have found the paper
		if len(approx) == 4:
			docCnt = approx
			break

   
	cv2.drawContours(output, [c], -1, (240, 0, 159), 3)
	#cv2.imshow("Contours", output)
	cv2.waitKey(0)

sử dụng perspective transform để kéo giãn khung bài trắc nghiệm.

 Trong trường hợp này chúng ta sẽ sử dụng hàm four_point_transform  với chức năng là:

    Xác định tọa độ (x, y)- contours với khả năng specific, reproducible manner.
    Ap dụng perspective transform cho các vùng.


In [62]:
# apply a four point perspective transform 
# to both the original image and grayscale image 
# to obtain a top-down birds eye view of the paper
paper 	= four_point_transform(image, docCnt.reshape(4, 2))

#cv2.imshow("Paper", paper)
cv2.waitKey(0)

-1

In [63]:
warped 	= four_point_transform(gray, docCnt.reshape(4, 2))

#cv2.imshow("Gray", warped)
cv2.waitKey(0)

-1

để đảm bảo ảnh không bị méo giống như ta dùng máy scan. 

Bước tiếp theo là chúng ta tiến hành nhị phân hóa ảnh :

In [64]:
# apply Otsu's thresholding method 
# to binarize the warped piece of paper
thresh = cv2.threshold(warped, 0, 255,
						cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]

#cv2.imshow("Thresh", thresh)
cv2.waitKey(0)

-1

 ảnh được nhị phân này sẽ giúp chúng ta sử dụng phép tìm contour 
 
 để tìm các khung tròn đáp án trên bài trắc nghiệm.

In [66]:
# find contours in the thresholded image, 
# then initialize the list of contours 
# that correspond to questions
cnts = cv2.findContours(thresh.copy(), 
                        cv2.RETR_EXTERNAL,
						cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
questionCnts = []   # danh sách contours tương ứng với các câu hỏi,trả lời/khoanh tròn trên bài trắc nghiệm.

paper2 = paper.copy()

khởi tạo questionCnts ,

nó chính là danh sách contours 

tương ứng với các câu hỏi,trả lời/khoanh tròn trên bài trắc nghiệm.

cần set điều kiện để kiểm tra xem contours đó có phải là vung tròn / câu trả lời của bài trắc nghiệm hay không:

    Chiều rộng và cao phải thích hợp như ở trong ví dụ này sẽ là > 20 pixels .
    Cần có tỉ lệ "aspect ratio"" xấp xỉ = 1.


In [35]:
# loop over the contours
for ques in cnts:
	# compute the bounding box of the contour, 
 	# then use the bounding box to derive the aspect ratio
	(x, y, w, h) = cv2.boundingRect(ques)  # Với mỗi contours, chúng ta tính bounding box
	ar = w / float(h)  					# tỉ lệ của chiều rộng với chiều cao 
 
	# in order to label the contour as a question, 
 	# region should be sufficiently wide, sufficiently tall, 
  	# and
	# have an aspect ratio approximately equal to 1
	if w >= 20 and h >= 20 and ar >= 0.9 and ar <= 1.1:
		questionCnts.append(ques)
  
		cv2.drawContours(paper2, ques, -1, (0, 255, 255), 3)
		#cv2.imshow("Question", paper2)
		cv2.waitKey(0)
		#cv2.destroyAllWindows()



In [50]:
cv2.destroyAllWindows()

In [ ]:
# sort the question contours top-to-bottom, 
# then initialize the total number of correct answers
questionCnts = contours.sort_contours(	questionCnts,
										method="top-to-bottom")[0]
correct = 0


In [ ]:
# each question has 5 possible answers, 
# to loop over the question 
# in batches of 5
for (q, i) in enumerate(np.arange(0, len(questionCnts), 5)):
	# sort the contours for the current question from
	# left to right, 
 	# then initialize the index of the
	# bubbled answer
	cnts = contours.sort_contours(questionCnts[i:i + 5])[0]
	bubbled = None
	#-------------------------------------------------------------------
 
 
	# loop over the sorted contours
	# Với mỗi hàng câu trả lời chúng ta tiến hành tìm kiếm các câu được trả lời bên trong ảnh.
	for (j, c) in enumerate(cnts):
		# construct a mask that reveals only the current
		# "bubble" for the question
		mask = np.zeros(thresh.shape, dtype="uint8")
		cv2.drawContours(mask, [c], -1, 255, -1)

		# apply the mask to the thresholded image, then
		# count the number of non-zero pixels in the
		# bubble area
		# sử dụng ảnh thresh  và đếm số lượng điểm ảnh có giá trị = 0 trên mỗi vùng khoanh tròn.
		mask 	= cv2.bitwise_and(thresh, thresh, mask=mask)
		total 	= cv2.countNonZero(mask)

		cv2.imshow("ms",mask)
		cv2.waitKey(0)

		# if the current total has a larger number of total
		# non-zero pixels, then we are examining the currently
		# bubbled-in answer
		if bubbled is None or total > bubbled[0]:
			bubbled = (total, j)

	
	# initialize the contour color and the index of the
	# *correct* answer
	color = (0, 0, 255)
	k = ANSWER_KEY[q]

	# check to see if the bubbled answer is correct
	if k == bubbled[1]:
		color = (0, 255, 0)
		correct += 1
	
	# draw the outline of the correct answer on the test
	cv2.drawContours(paper, [cnts[k]], -1, color, 3)
	cv2.imshow("Ex",paper)
	cv2.waitKey(0)

In [ ]:
# grab the test taker
score = (correct / 5.0) * 100
print("[INFO] score: {:.2f}%".format(score))

cv2.putText(paper, 
            "{:.2f}%".format(score), 
            (10, 30),
			cv2.FONT_HERSHEY_SIMPLEX, 
   			0.9, (0, 0, 255), 2)
cv2.imshow("Original", image)
cv2.imshow("Exam", paper)

cv2.imwrite("/home/anhp/Documents/coding/learnOpenCV/BubbleSheetScanner/exa/Exam1_1.png", paper)
cv2.waitKey(0)
cv2.destroyAllWindows()